In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np

from attila.data.parse import parse_data, get_data
from attila.experiments.do import do_batch_experiments, do_experiment
from attila.util.config import get_env
from attila.util.io import load_json, stuff2pickle
from attila.util.plots import extract_preds

Using TensorFlow backend.


In [3]:
_here = Path('.').resolve()

In [4]:
config, data_path, out_path, _ = get_env(_here)
out_path.mkdir(parents=True, exist_ok=True)  # rm and mkdir if existing

images_path = data_path / config.get('data', 'images')
masks_path = data_path / config.get('data', 'masks')

In [5]:
raw = get_data(images_path, masks_path)
X, y = parse_data(
    raw,
    (config.getint('image', 'width'), config.getint('image', 'height'))
)

In [6]:
config.set('experiments', 'test size', '0.95')  # or any other big amount (< 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=config.getfloat('experiments', 'test size'),
    random_state=42  # reproducible results
)
print('train/val data: X ~ {}, y ~ {}'.format(X_train.shape, y_train.shape))
print('test data: X ~ {}, y ~ {}'.format(X_test.shape, y_test.shape))

num_plots = 4
plot_ids = np.random.randint(len(X_test), size=num_plots)

train/val data: X ~ (4, 512, 512, 1), y ~ (4, 512, 512, 3)
test data: X ~ (88, 512, 512, 1), y ~ (88, 512, 512, 3)


In [7]:
experiment = {
  "use_skip_conn": True,
  "padding": "same",
  "use_se_block": False,
  "name": "with_same"
}

config.set('experiments', 'val size', '0.5')  # not very necessary to validate
config.set('training', 'batch size', '2')  # very mini-batch size
config.set('training', 'epochs', '20')  # or any other small amount

In [8]:
config.set('data', 'aug', 'False')

summary = do_experiment(experiment, (X_train, X_test, y_train, y_test), 0, config, plot_ids)

out_folder = out_path / 'trials' / 'no-aug'
out_folder.mkdir(parents=True, exist_ok=True)
out_f = out_folder / config.get('experiments', 'output file')
stuff2pickle(summary, out_f)

training data: X ~ (4, 512, 512, 1), y ~ (4, 512, 512, 3)
testing data: X ~ (88, 512, 512, 1), y ~ (88, 512, 512, 3)
Epoch 1/20
8/8 [==============================] - 14s 2s/step - loss: 1.0859 - accuracy: 0.0901 - attila_metrics_mean_IoU: 0.0356 - attila_metrics_DSC: 0.0653 - val_loss: 1.0760 - val_accuracy: 0.2682 - val_attila_metrics_mean_IoU: 0.0406 - val_attila_metrics_DSC: 0.0733 - lr: 2.0000e-05
Epoch 2/20
8/8 [==============================] - 13s 2s/step - loss: 1.0715 - accuracy: 0.6942 - attila_metrics_mean_IoU: 0.0356 - attila_metrics_DSC: 0.0653 - val_loss: 1.0599 - val_accuracy: 0.9434 - val_attila_metrics_mean_IoU: 0.0406 - val_attila_metrics_DSC: 0.0733 - lr: 2.0000e-05
Epoch 3/20
8/8 [==============================] - 13s 2s/step - loss: 1.0508 - accuracy: 0.9617 - attila_metrics_mean_IoU: 0.0356 - attila_metrics_DSC: 0.0653 - val_loss: 1.0236 - val_accuracy: 0.9751 - val_attila_metrics_mean_IoU: 0.0406 - val_attila_metrics_DSC: 0.0733 - lr: 2.0000e-05
Epoch 4/20
8/8 [

In [9]:
config.set('data', 'aug', 'True')

summary = do_experiment(experiment, (X_train, X_test, y_train, y_test), 0, config, plot_ids)

out_folder = out_path / 'trials' / 'aug'
out_folder.mkdir(parents=True, exist_ok=True)
out_f = out_folder / config.get('experiments', 'output file')
stuff2pickle(summary, out_f)

training data: X ~ (4, 512, 512, 1), y ~ (4, 512, 512, 3)
testing data: X ~ (88, 512, 512, 1), y ~ (88, 512, 512, 3)
Epoch 1/20
8/8 [==============================] - 14s 2s/step - loss: 1.0879 - accuracy: 0.2905 - attila_metrics_mean_IoU: 0.0356 - attila_metrics_DSC: 0.0653 - val_loss: 1.0753 - val_accuracy: 0.8413 - val_attila_metrics_mean_IoU: 0.0406 - val_attila_metrics_DSC: 0.0733 - lr: 2.0000e-05
Epoch 2/20
8/8 [==============================] - 13s 2s/step - loss: 1.0683 - accuracy: 0.9301 - attila_metrics_mean_IoU: 0.0356 - attila_metrics_DSC: 0.0653 - val_loss: 1.0540 - val_accuracy: 0.9668 - val_attila_metrics_mean_IoU: 0.0406 - val_attila_metrics_DSC: 0.0733 - lr: 2.0000e-05
Epoch 3/20
8/8 [==============================] - 13s 2s/step - loss: 1.0437 - accuracy: 0.9748 - attila_metrics_mean_IoU: 0.0356 - attila_metrics_DSC: 0.0653 - val_loss: 1.0132 - val_accuracy: 0.9837 - val_attila_metrics_mean_IoU: 0.0406 - val_attila_metrics_DSC: 0.0733 - lr: 2.0000e-05
Epoch 4/20
8/8 [

KeyboardInterrupt: 